In [380]:
import pandas as pd
import numpy as np
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError

In [381]:
train = pd.read_csv('Data/sales_train_evaluation.csv')
cal = pd.read_csv('Data/calendar.csv')
prices = pd.read_csv('Data/sell_prices.csv')
predictions = pd.read_csv('Data/predictions.csv')

In [382]:
train

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [383]:
predictions

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.862952,0.705364,0.662255,0.656420,0.688944,0.859311,0.952116,0.778295,0.895433,...,0.873363,1.260613,1.027292,0.850092,0.740439,0.767691,0.953557,0.887824,1.057381,1.079290
1,HOBBIES_1_002_CA_1_evaluation,0.402444,0.345715,0.345715,0.320039,0.338185,0.393470,0.395760,0.269730,0.226057,...,0.200355,0.267870,0.265104,0.195934,0.166295,0.224661,0.209601,0.242574,0.297260,0.294056
2,HOBBIES_1_003_CA_1_evaluation,0.484242,0.456097,0.456097,0.456097,0.489418,0.544687,0.653748,0.396880,0.344197,...,0.540993,0.682991,0.905492,0.721303,0.710370,0.715409,0.658388,0.689722,0.878399,0.821469
3,HOBBIES_1_004_CA_1_evaluation,2.041641,1.707082,1.533395,1.448839,1.742334,1.977546,1.909560,1.485940,1.686545,...,1.853893,2.066903,2.402056,1.907649,1.681269,1.621097,1.619189,1.860526,2.828348,2.534054
4,HOBBIES_1_005_CA_1_evaluation,0.883296,0.761569,0.990771,1.185331,1.109823,1.453893,1.791780,1.312651,1.247941,...,1.167325,1.422263,1.411676,1.005493,0.986156,0.970658,0.985911,1.057604,1.337586,1.638328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.307081,0.272211,0.272211,0.272211,0.291866,0.318660,0.379767,0.384343,0.348262,...,0.491590,0.501441,0.519493,0.334687,0.287529,0.287529,0.330749,0.304152,0.344421,0.420380
30486,FOODS_3_824_WI_3_evaluation,0.763628,0.742199,0.742199,0.742199,0.882743,0.645386,0.361743,0.428425,0.359154,...,0.347587,0.398658,0.388152,0.249185,0.169004,0.169004,0.168804,0.194905,0.304184,0.275971
30487,FOODS_3_825_WI_3_evaluation,0.785894,0.729165,0.729165,0.686009,0.661727,0.830718,0.874793,0.826966,0.794378,...,0.969824,1.075831,1.008434,0.820193,0.718267,0.750560,0.728368,0.741478,0.877860,0.849646
30488,FOODS_3_826_WI_3_evaluation,1.268862,1.187438,1.152862,1.071365,1.114729,1.241293,1.258717,1.094381,1.113772,...,1.093538,1.235366,1.197084,0.966669,0.846541,0.848843,0.815386,1.089504,1.179069,1.335390


In [384]:
def revert_cols(predictions):
    parts = predictions['id'].str.split(pat="_",expand=True)
    predictions_full = pd.DataFrame(data={'id':predictions['id']})
    predictions_full['item_id'] = parts[0] + '_' + parts[1] + '_' + parts[2]
    predictions_full['dept_id'] = parts[0] + '_' + parts[1]
    predictions_full['cat_id'] = parts[0]
    predictions_full['store_id'] = parts[3] + '_' + parts[4]
    predictions_full['state_id'] = parts[3]
    predictions_full = pd.concat([predictions_full,predictions.iloc[:,1:]],axis=1)
    return predictions_full

In [385]:
predictions_full = revert_cols(predictions)

In [386]:
predictions_full

,id,item_id,dept_id,cat_id,store_id,state_id,F1,F2,F3,F4,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.862952,0.705364,0.662255,0.656420,...,0.873363,1.260613,1.027292,0.850092,0.740439,0.767691,0.953557,0.887824,1.057381,1.079290
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.402444,0.345715,0.345715,0.320039,...,0.200355,0.267870,0.265104,0.195934,0.166295,0.224661,0.209601,0.242574,0.297260,0.294056
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.484242,0.456097,0.456097,0.456097,...,0.540993,0.682991,0.905492,0.721303,0.710370,0.715409,0.658388,0.689722,0.878399,0.821469
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2.041641,1.707082,1.533395,1.448839,...,1.853893,2.066903,2.402056,1.907649,1.681269,1.621097,1.619189,1.860526,2.828348,2.534054
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.883296,0.761569,0.990771,1.185331,...,1.167325,1.422263,1.411676,1.005493,0.986156,0.970658,0.985911,1.057604,1.337586,1.638328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0.307081,0.272211,0.272211,0.272211,...,0.491590,0.501441,0.519493,0.334687,0.287529,0.287529,0.330749,0.304152,0.344421,0.420380
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0.763628,0.742199,0.742199,0.742199,...,0.347587,0.398658,0.388152,0.249185,0.169004,0.169004,0.168804,0.194905,0.304184,0.275971
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0.785894,0.729165,0.729165,0.686009,...,0.969824,1.075831,1.008434,0.820193,0.718267,0.750560,0.728368,0.741478,0.877860,0.849646
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1.268862,1.187438,1.152862,1.071365,...,1.093538,1.235366,1.197084,0.966669,0.846541,0.848843,0.815386,1.089504,1.179069,1.335390


In [387]:
def create_levels(df):
    #total
    full_wlevels = pd.DataFrame(data={'level':['total'],'id':['01_total'],'item_id':['total'],'dept_id':['total'],'cat_id':['total'],'store_id':['total'],'state_id':['total']})
    full_wlevels = pd.concat([full_wlevels,df.sum(numeric_only=True).to_frame().T],axis=1)
    #state
    new_level = df.groupby('state_id').sum().reset_index()
    new_level['level'] = 'state'
    new_level['id'] = '02_' + new_level['state_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #store
    new_level = df.groupby('store_id').sum().reset_index()
    new_level['level'] = 'store'
    new_level['id'] = '03_' + new_level['store_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #category
    new_level = df.groupby('cat_id').sum().reset_index()
    new_level['level'] = 'category'
    new_level['id'] = '04_' + new_level['cat_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #department
    new_level = df.groupby('dept_id').sum().reset_index()
    new_level['level'] = 'department'
    new_level['id'] = '05_' + new_level['dept_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #state_category
    new_level = df.groupby(['state_id','cat_id']).sum().reset_index()
    new_level['level'] = 'state_category'
    new_level['id'] = '06_' + new_level['state_id'] + '_' +new_level['cat_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #state_department
    new_level = df.groupby(['state_id','dept_id']).sum().reset_index()
    new_level['level'] = 'state_department'
    new_level['id'] = '07_' + new_level['state_id'] + '_' +new_level['dept_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #store_category
    new_level = df.groupby(['store_id','cat_id']).sum().reset_index()
    new_level['level'] = 'store_category'
    new_level['id'] = '08_' + new_level['store_id'] + '_' +new_level['cat_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #store_department
    new_level = df.groupby(['store_id','dept_id']).sum().reset_index()
    new_level['level'] = 'store_department'
    new_level['id'] = '09_' + new_level['store_id'] + '_' +new_level['dept_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #item
    new_level = df.groupby('item_id').sum().reset_index()
    new_level['level'] = 'item'
    new_level['id'] = '10_' + new_level['item_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #item_state
    new_level = df.groupby(['item_id','state_id']).sum().reset_index()
    new_level['level'] = 'item_state'
    new_level['id'] = '11_' + new_level['item_id'] + '_' + new_level['state_id']
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #item_store
    df['level'] = 'item_store'
    full_wlevels = pd.concat([full_wlevels,df],axis=0)
    #return final df
    return full_wlevels.sort_values(by='id')

In [388]:
full_wlevels = create_levels(train).reset_index(drop=True)
predictions_full_wlevels = create_levels(predictions_full).reset_index(drop=True)

In [389]:
predictions_full_wlevels

,level,id,item_id,dept_id,cat_id,store_id,state_id,F1,F2,F3,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,total,01_total,total,total,total,total,total,37212.664918,35163.199726,34921.469686,...,42949.593258,51087.016163,52007.355862,39391.736256,36269.394887,36094.774193,35940.258471,40387.183636,49001.445863,49142.212383
1,state,02_CA,NaN,NaN,NaN,NaN,CA,16111.400816,15185.827157,15070.002990,...,18072.923527,21783.212538,22251.057719,16979.391767,15598.483833,15527.583128,15418.572354,17353.080658,21488.856550,21683.505022
2,state,02_TX,NaN,NaN,NaN,NaN,TX,10307.481299,9539.795755,9481.099835,...,11495.184365,13368.250434,13690.829380,10615.548896,9767.655056,9693.754114,9656.944532,10838.751128,12963.808272,12923.988520
3,state,02_WI,NaN,NaN,NaN,NaN,WI,10793.782803,10437.576815,10370.366862,...,13381.485366,15935.553191,16065.468763,11796.795592,10903.255998,10873.436951,10864.741584,12195.351850,14548.781041,14534.718841
4,store,03_CA_1,NaN,NaN,NaN,CA_1,NaN,4024.161970,3801.930958,3754.641748,...,4546.410549,5584.979672,5717.938955,4234.226391,3881.881569,3866.552661,3860.227527,4386.004988,5472.248647,5561.245551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42835,item_store,HOUSEHOLD_2_516_TX_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,TX_2,TX,0.255847,0.227702,0.276895,...,0.271006,0.411340,0.427226,0.277514,0.230517,0.273937,0.259076,0.309434,0.339591,0.270074
42836,item_store,HOUSEHOLD_2_516_TX_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,TX_3,TX,0.130741,0.164760,0.180753,...,0.820888,0.154911,0.220457,0.255292,0.249747,0.249747,0.249747,0.314928,0.355323,0.355323
42837,item_store,HOUSEHOLD_2_516_WI_1_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_1,WI,0.094888,0.073459,0.073459,...,0.173861,0.167227,0.169993,0.139781,0.106507,0.106507,0.106507,0.099584,0.126379,0.126379
42838,item_store,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,0.092361,0.070933,0.070933,...,0.118363,0.145491,0.148258,0.110531,0.083973,0.083973,0.118841,0.134832,0.161626,0.161626


In [390]:
# define your train and test dataset here
features = 6
train_end = 1919
test_end = 1947

In [404]:
def calculate_naive_mse(series):
    start = 0
    naive_mse = 0
    for i in range(len(series)):
        if start == 0 and series[i]>0:
            start = i
            #print(start)
        if i > start:
            new_se = (series[i]-series[i-1])**2
            naive_mse += new_se
        end = i
    #print(end)
    naive_mse = naive_mse / (end-start)
    return naive_mse

def get_naive_mse(full_wlevels,features,train_end):
    trainf = full_wlevels.iloc[:,features+1:train_end+1]
    naive_mse = pd.DataFrame(data={'id':full_wlevels['id'],'naive_mse':trainf.apply(calculate_naive_mse, axis=1)})
    return naive_mse

In [405]:
naive_mse = get_naive_mse(full_wlevels,features,train_end)

In [406]:
naive_mse

,id,naive_mse
0,01_total,3.513560e+07
1,02_CA,7.345027e+06
2,02_TX,3.341276e+06
3,02_WI,3.767079e+06
4,03_CA_1,7.503178e+05
...,...,...
42835,HOUSEHOLD_2_516_TX_2_evaluation,6.190476e-01
42836,HOUSEHOLD_2_516_TX_3_evaluation,2.249737e-01
42837,HOUSEHOLD_2_516_WI_1_evaluation,1.712939e-01
42838,HOUSEHOLD_2_516_WI_2_evaluation,6.986667e-02


In [399]:
def get_weights(train,cal,prices):    
    test_weights = train.iloc[:,np.r_[:features,train_end:test_end]]
    test_weights = pd.melt(test_weights, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    cal_M = cal[['wm_yr_wk','d']]
    test_weights = pd.merge(test_weights, cal_M, how = 'left', left_on = ['day'], right_on = ['d']).drop('d', axis=1)
    test_weights = test_weights.merge(prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    test_weights['sales'] = test_weights['demand']*test_weights['sell_price']
    test_weights = test_weights.drop(['demand','wm_yr_wk','sell_price'],axis=1)
    test_weights = test_weights.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','day'])['sales'].unstack().reset_index()
    test_weights_wlevels = create_levels(test_weights)
    total_sales = float(test_weights_wlevels[test_weights_wlevels['level']=='total'].sum(numeric_only=True,axis=1))
    test_weights_wlevels['weights'] = test_weights_wlevels.sum(numeric_only=True,axis=1)/total_sales
    return test_weights_wlevels[['id','weights']]

In [400]:
weights = get_weights(train,cal,prices)

In [401]:
weights

,id,weights
0,01_total,1.000000
0,02_CA,0.441376
1,02_TX,0.275960
2,02_WI,0.282664
0,03_CA_1,0.111262
...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.000006
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.000012
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.000008
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.000003


In [407]:
def get_scores(predictions_full_wlevels,full_wlevels,naive_mse,weights):
    yhat_matrix = predictions_full_wlevels.iloc[:,features+1:].to_numpy()
    y_matrix = full_wlevels.iloc[:,train_end+1:test_end+1].to_numpy()
    predictions_wscores = predictions_full_wlevels.copy()
    errors_matrix = yhat_matrix - y_matrix
    predictions_wscores['totalerror'] = errors_matrix.sum(axis=1)
    sqerror_matrix = errors_matrix**2
    predictions_wscores['totalsqerror'] = sqerror_matrix.sum(axis=1)
    predictions_wscores['rmse'] = (sqerror_matrix.sum(axis=1) / sqerror_matrix.shape[1])**.5
    predictions_wscores['rmsse'] = (sqerror_matrix.sum(axis=1) / naive_mse['naive_mse'] / sqerror_matrix.shape[1])**.5
    predictions_wscores['weights'] = list(weights['weights'])
    predictions_wscores['wrmse'] = predictions_wscores['rmse'] * predictions_wscores['weights']
    predictions_wscores['wrmsse'] = predictions_wscores['rmsse'] * predictions_wscores['weights']
    predictions_wscores['twrmse'] = predictions_wscores['wrmse'] / 12
    predictions_wscores['twrmsse'] = predictions_wscores['wrmsse'] / 12
    return predictions_wscores

In [408]:
predictions_wscores = get_scores(predictions_full_wlevels,full_wlevels,naive_mse,weights)

In [409]:
predictions_wscores.groupby('level').sum().iloc[:,28:].sort_values(by='twrmsse')

,totalerror,totalsqerror,rmse,rmsse,weights,wrmse,wrmsse,twrmse,twrmsse
level,,,,,,,,,
total,-53920.980406,2.155232e+08,2774.393298,0.468052,1.0,2774.393298,0.468052,231.199442,0.039004
category,-53920.980406,1.525587e+08,3144.846595,1.437722,1.0,1504.336243,0.503206,125.361354,0.041934
state,-53920.980406,1.066883e+08,3372.383270,1.607469,1.0,1123.137700,0.515827,93.594808,0.042986
state_category,-53920.980406,7.746764e+07,3949.942716,5.255584,1.0,616.938528,0.569997,51.411544,0.047500
department,-53920.980406,8.442488e+07,3642.086711,5.251625,1.0,784.921418,0.583345,65.410118,0.048612
store,-53920.980406,5.667271e+07,4286.530814,6.353627,1.0,443.383759,0.630317,36.948647,0.052526
state_department,-53920.980406,4.423901e+07,4550.117000,16.622126,1.0,329.721727,0.638945,27.476811,0.053245
store_category,-53920.980406,4.079988e+07,5097.932104,20.383189,1.0,246.077660,0.673639,20.506472,0.056137
store_department,-53920.980406,2.553510e+07,6021.642856,57.756693,1.0,140.539919,0.736139,11.711660,0.061345


In [410]:
predictions_wscores['twrmsse'].sum()

0.6757954099347587